#**TITANIC DATA TO DETERMINE WHETHERE THE PASSENGER IS SURVIVED OR NOT**

In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Importing the data
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

#checking the data to find null values
train.info(),test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

(None, None)

##**ANALYSE THE DATA**

In [21]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


##**ENCODING THE DATA TO USE IT FOR CLASSIFIER**

In [23]:
train['Sex']=(train['Sex']=='male').astype('int')
test['Sex']=(test['Sex']=='male').astype('int')

model=LabelEncoder().fit(train['Embarked'].tolist())
train['Embarked']=model.transform(train['Embarked'].tolist())
test['Embarked']=model.transform(test['Embarked'].tolist())

#Removing the Sibsp ,parch and merging it into a single column as family
train['Family']=train['SibSp']+train['Parch']
test['Family']=test['SibSp']+test['Parch']
end=test['PassengerId']

#we can remove data which is not needed for both the data
train.drop(['PassengerId','Name','Ticket','Cabin','Parch','SibSp'],inplace=True,axis=1)
test.drop(['PassengerId','Name','Ticket','Cabin','SibSp','Parch'],inplace=True,axis=1)
y=train['Survived']
train.drop(['Survived'],inplace=True,axis=1)

In [24]:
train.head()

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,1,22.0,7.2500,2,1
1,1,0,38.0,71.2833,0,1
2,3,0,26.0,7.9250,2,0
3,1,0,35.0,53.1000,2,1
4,3,1,35.0,8.0500,2,0


In [25]:
test.head()

,Pclass,Sex,Age,Fare,Embarked,Family
0,3,1,34.5,7.8292,1,0
1,3,0,47.0,7.0000,2,1
2,2,1,62.0,9.6875,1,0
3,3,1,27.0,8.6625,2,0
4,3,0,22.0,12.2875,2,2


##**TO DETERMINE THE AGE FOR SOME NULL VALUES**

In [26]:
check=train.groupby(['Pclass','Sex'])['Age'].apply(np.mean).to_frame()
check

Age
Pclass Sex           
1      0    34.611765
       1    41.281386
2      0    28.722973
       1    30.740707
3      0    21.750000
       1    26.507589

In [27]:
for i in train[train['Age'].isna()].index:
  train.loc[i,'Age']=int(check.loc[train.iloc[i]['Pclass'],train.iloc[i]['Sex']])

for i in test[test['Age'].isna()].index:
  test.loc[i,'Age']=int(check.loc[test.iloc[i]['Pclass'],test.iloc[i]['Sex']])


##**TO DETERMINE THE FARE FOR ONE OF THE NULL VALUES**

In [28]:
train.groupby(['Embarked','Pclass'])['Fare'].apply(np.mean)

Embarked  Pclass
0         1         104.718529
          2          25.358335
          3          11.214083
1         1          90.000000
          2          12.350000
          3          11.183393
2         1          70.364862
          2          20.327439
          3          14.644083
3         1          80.000000
Name: Fare, dtype: float64

In [29]:
test.loc[test['Fare'].isna(),'Fare']=14.644

In [30]:
test.isna().sum()

Pclass      0
Sex         0
Age         0
Fare        0
Embarked    0
Family      0
dtype: int64

##**CREATING A SET OF RANGE OF VALUES (AGE)**

In [31]:
waste,cuts=pd.cut(train['Age'],5,retbins=True)
cuts[0]=0

train['Age']=pd.cut(train['Age'],cuts)
test['Age']=pd.cut(test['Age'],cuts)
model=LabelEncoder().fit(train['Age'])
train['Age']=model.transform(train['Age'])
test['Age']=model.transform(test['Age'])

##**MODEL SELECTION AND IMPLEMENTATION**

In [32]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression().fit(train,y)
model.score(train,y)

0.8058361391694725

In [33]:
model.predict(test)

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [39]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=0).fit(train,y)
model.score(train,y)

0.9405162738496072

In [40]:
model.predict(test)

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [41]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier(random_state=0).fit(train,y)
model.score(train,y)

0.9405162738496072

In [42]:
result=pd.DataFrame(model.predict(test),index=end,columns=['Survived'])

In [43]:
result

,Survived
PassengerId,
892,0
893,1
894,1
895,0
896,1
...,...
1305,0
1306,1
1307,0


In [ ]:
result.to_csv('result.csv')